# Political Organizations Activity on Twitter aggregated by Party

The parameters in the cell below can be adjusted to explore other political parties and time frames.

### How to explore other political parties?
The ***organization*** parameter can be use to aggregate organizations by their party. The column `subcategory` in this [this other notebook](../political_organizations.ipynb?autorun=true) show the organizations that belong each party.

***Alternatively***, you can direcly use the [organizations API](http://mediamonitoring.gesis.org/api/organizations/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

## A. Set Up parameters

In [ ]:
# Parameters: 
organization = 'csu'
from_date = '2017-09-01'
to_date = '2018-12-31'
aggregation = 'week'

## B. Using the SMM Politician API

In [ ]:
import pandas as pd

# create an instance to the smm wrapper
from smm_wrapper import SMMOrganizations
smm = SMMOrganizations()

# using the api to get the data
df = smm.dv.get_organizations()
df = df[(df['category']=='political')]

# filter the accounts by party, and valid ones (the ones that contain tw_ids)
party_df = df[(df['subcategory']==organization) & (df['tw_ids'].notnull())] 

# query the Social Media Monitoring API
tweets_by = pd.concat(smm.dv.tweets_by(_id=organization_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for organization_id in party_df.index)
replies_to = pd.concat(smm.dv.replies_to(_id=organization_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for organization_id in party_df.index)

# aggregate the replies
total_tweets_by = tweets_by.groupby('date').agg({'tweets': 'sum'})
total_replies_to = replies_to.groupby('date').agg({'replies': 'sum'})

## C. Plotting

In [ ]:
#plotting data
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=total_tweets_by.index, y=total_tweets_by['tweets'], name='Tweets', line_shape='spline'), 
             go.Scatter(x=total_replies_to.index, y=total_replies_to['replies'], name='Replies', line_shape='spline')], 
    "layout": go.Layout( title='Twitter (Tweets and Replies)', yaxis=dict(title='N'))
})